In [ ]:
import pathlib
import csv
import re

from readers.pipelines import subject, probe, session
from readers.paths import get_ephys_root_data_dir
from readers import intantech

from readers import ephys
import readers.utils

def ingest_subjects(subject_csv_path='./user_data/subjects.csv'):
    # -------------- Insert new "Subject" --------------
    with open(subject_csv_path, newline= '') as f:
        input_subjects = list(csv.DictReader(f, delimiter=','))

    print(f'\n---- Insert {len(input_subjects)} entry(s) into subject.Subject ----')
    subject.Subject.insert(input_subjects, skip_duplicates=True)


def ingest_sessions(session_csv_path='./user_data/sessions.csv'):
    root_data_dir = get_ephys_root_data_dir()

    # ---------- Insert new "Session" and "ProbeInsertion" ---------
    with open(session_csv_path, newline= '') as f:
        input_sessions = list(csv.DictReader(f, delimiter=','))

    # Folder structure: root / subject / session / probe / .ap.meta
    session_list, session_dir_list, probe_list, probe_insertion_list = [], [], [], []

    for sess in input_sessions:
        session_dir = readers.utils.find_full_path(
                                                    get_ephys_root_data_dir(), 
                                                    sess['session_dir'])
        session_datetimes, insertions = [], []

        # search session dir and determine acquisition software
        for ephys_pattern, ephys_acq_type in zip(['*.ap.meta', '*.oebin'], ['IntanTECH']):
            ephys_meta_filepaths = [fp for fp in session_dir.rglob(ephys_pattern)]
            if len(ephys_meta_filepaths):
                acq_software = ephys_acq_type
                break
        else:
            raise FileNotFoundError(f'Ephys recording data not found! No IntanTECH recording files found in: {session_dir}')

        if acq_software == 'IntanTECH':
            for meta_filepath in ephys_meta_filepaths:
               intantech_meta = intantech.IntanTECHMeta(meta_filepath)
               probe_key = {'probe_type': intantech_meta.probe_model, 'probe': intantech_meta.probe_SN}
               if probe_key['probe'] not in [p['probe'] for p in probe_list] and probe_key not in probe.Probe():
                    probe_list.append(probe_key)
                    
                    probe_dir = meta_filepath.parent
                    probe_number = re.search('(imec)?\d{1}$', probe_dir.name).group()
                    probe_number = int(probe_number.replace('imec', ''))

                    insertions.append({'probe': intantech_meta.probe_SN, 'insertion_number': int(probe_number)})
                    session_datetimes.append(intantech_meta.recording_time)

        else:
            raise NotImplementedError(f'Unknown acquisition software: {acq_software}')

        # new session/probe-insertion
        session_key = {'subject': sess['subject'], 'session_datetime': min(session_datetimes)}
        if session_key not in session.Session():
            session_list.append(session_key)
            session_dir_list.append({**session_key, 'session_dir': session_dir.relative_to(root_data_dir).as_posix()})
            probe_insertion_list.extend([{**session_key, **insertion} for insertion in insertions])

    print(f'\n---- Insert {len(session_list)} entry(s) into session.Session ----')
    session.Session.insert(session_list)
    session.SessionDirectory.insert(session_dir_list)

    print(f'\n---- Insert {len(probe_list)} entry(s) into probe.Probe ----')
    probe.Probe.insert(probe_list)

    print(f'\n---- Insert {len(probe_insertion_list)} entry(s) into ephys.ProbeInsertion ----')
    ephys.ProbeInsertion.insert(probe_insertion_list)

    print('\n---- Successfully completed element_array_ephys/ingest.py ----')

if __name__ == '__main__':
    ingest_subjects()
    ingest_sessions()